In [1]:
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
)
from codebase.plot import plot_density, plot_histogram, plot_line, get_post_df
from codebase.ibis import post_process_sign, exp_and_normalise
import altair as alt
from codebase.classes_data import Data
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from pdb import set_trace

alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [41]:
log_dir1 = 'log/20210308_215813_mcmc_rasch/'
exp_data1 = load_obj( 'complete_data', log_dir1)
ps_mcmc = load_obj('mcmc_post_samples', log_dir1)


In [42]:
log_dir3 = 'log/20210411_144529_lvm_prior/'
exp_data3 = load_obj('data', log_dir3)
t = load_obj('t', log_dir3)
print(t)


38


In [43]:
particles = load_obj('particles', log_dir3)
ps_smclvm = particles.particles
# ps_smc2 = dict()
# for name in particles.param_names:
#     ps_smc2[name] = particles.extract_particles_in_numpy_array(name)
# particles.weights

In [44]:
particles.get_threshold_ess_indicator().sum()

18

In [45]:
particles.get_threshold_ess_indicator()

array([False,  True, False,  True, False,  True, False, False,  True,
       False,  True,  True,  True,  True, False,  True, False,  True,
       False, False,  True, False,  True, False, False,  True, False,
       False, False,  True, False,  True, False,  True, False,  True,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [46]:
load_obj('jitter_corrs', log_dir3)

{0: {'alpha': array([1., 1., 1., 1., 1., 1.]), 'beta': array([[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]]), 'zt': array([1.]), 'yt': array([1., 1., 1., 1., 1., 1.])},
 1: {'alpha': array([1., 1., 1., 1., 1., 1.]), 'beta': array([[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]]), 'zt': array([1.]), 'yt': array([1., 1., 1., 1., 1., 1.])},
 2: {},
 3: {'alpha': array([1., 1., 1., 1., 1., 1.]), 'beta': array([[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]]), 'zt': array([1.]), 'yt': array([1., 1., 1., 1., 1., 1.])},
 4: {},
 5: {'alpha': array([1., 1., 1., 1., 1., 1.]), 'beta': array([[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]]), 'zt': array([1.]), 'yt': array([1., 1., 1., 1., 1., 1.])},
 6: {},
 7: {},
 8: {'alpha': array([1., 1., 1., 1., 1., 1.]), 'beta': array([[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]]), 'zt': a

In [47]:
# particles = load_obj('particles', log_dir3)
particles.resample_particles()
# ps_smc2 = dict()
# for name in particles.param_names:
#     ps_smc2[name] = particles.extract_particles_in_numpy_array(name)


##

## Or load existing directory

In [48]:
ps_mcmc = post_process_sign(ps_mcmc)
ps_smclvm = post_process_sign(ps_smclvm)


## Plot MCMC samples

In [49]:
param = 'beta'
df = get_post_df(ps_mcmc[param]) 
df = df[df.row==0]
df['source'] = 'mcmc'


df3 = get_post_df(ps_smclvm[param]) 
df3 = df3[df3.row==0]
df3['source'] = 'ibis'

plot_density(pd.concat([df, df3]), height=100) 

alt.Chart(...)

In [50]:
param = 'alpha'
df = get_post_df(ps_mcmc[param]) 
df['source'] = 'mcmc'


df3 = get_post_df(ps_smclvm[param]) 
df3['source'] = 'ibis'

plot_density(pd.concat([df, df3]), height=100)

alt.Chart(...)

## Plot quantiles

In [39]:
param = 'beta'
df = get_post_df(ps_smclvm[param])
df_quant = df.groupby(['row', 'col'])[['value']].quantile(0.025).reset_index()
df_quant.rename({'value':'q1'}, axis=1, inplace=True)
df_quant2 = df.groupby(['row', 'col'])[['value']].quantile(0.975).reset_index()
df_quant2.rename({'value':'q2'}, axis=1, inplace=True)

df = df_quant.merge(df_quant2, on=['row', 'col'])

# simple quantile chart
df['source'] = 'ibis'
c1 = alt.Chart(df[df.row==0]).mark_bar(opacity=0.6).encode(
        alt.X('q1', title=None),
        alt.X2('q2', title=None),
        alt.Row('row'),
        alt.Column('col'),
        alt.Color('source')
)
c1

alt.Chart(...)

## chart overlaying quantiles with real data

In [23]:
df['index'] = 'r_' + df.row.astype(str)+'.c_'+df.col.astype(str)
df = df.loc[:,['index', 'q1', 'q2']]
df['source'] = 'ibis'
dd = pd.DataFrame(exp_data1.raw_data['beta'], columns=['data'])
dd['col'] = 0
dd['row'] = np.arange(6)
dd['index'] = 'r_' + dd.row.astype(str)+'.c_'+dd.col.astype(str)
dd = dd.loc[:,['index', 'data']]
plot_data = df.merge(dd, on=['index'])

In [24]:
plot_data = plot_data[plot_data['index']=='r_0.c_0']

c1 = alt.Chart(plot_data).mark_bar(opacity=0.6).encode(
    alt.X('q1', title=None,  scale=alt.Scale(domain=[-2,2])),
    alt.X2('q2', title=None),
    alt.Color('source'),    
)
    

c2 = alt.Chart(plot_data).mark_point(opacity=1, color='red').encode(
        alt.X('data', title=None),
)
(c1+c2).facet(
       'index',
    columns=1
    )


alt.FacetChart(...)


## chart overlaying quantiles with real data

In [25]:
param = 'alpha'
df = get_post_df(ps_smclvm[param])
df_quant = df.groupby(['row', 'col'])[['value']].quantile(0.025).reset_index()
df_quant.rename({'value':'q1'}, axis=1, inplace=True)
df_quant2 = df.groupby(['row', 'col'])[['value']].quantile(0.975).reset_index()
df_quant2.rename({'value':'q2'}, axis=1, inplace=True)

df = df_quant.merge(df_quant2, on=['row', 'col'])

# simple quantile chart
df['source'] = 'ibis'
c1 = alt.Chart(df).mark_bar(opacity=0.6).encode(
        alt.X('q1', title=None),
        alt.X2('q2', title=None),
        alt.Row('row'),
        alt.Column('col'),
        alt.Color('source')
)
c1

alt.Chart(...)

In [26]:
df['index'] = 'r_' + df.row.astype(str)+'.c_'+df.col.astype(str)
df = df.loc[:,['index', 'q1', 'q2']]
df['source'] = 'ibis'
dd = pd.DataFrame(exp_data1.raw_data['alpha'], columns=['data'])
dd['col'] = np.arange(6)
dd['row'] = 0
dd['index'] = 'r_' + dd.row.astype(str)+'.c_'+dd.col.astype(str)
dd = dd.loc[:,['index', 'data']]
plot_data = df.merge(dd, on=['index'])

In [27]:
c1 = alt.Chart(plot_data).mark_bar(opacity=0.6).encode(
    alt.X('q1', title=None,  scale=alt.Scale(domain=[-5,5])),
    alt.X2('q2', title=None),
    alt.Color('source'),    
)
    

c2 = alt.Chart(plot_data).mark_point(opacity=1, color='red').encode(
        alt.X('data', title=None),
)
(c1+c2).facet(
       'index',
    columns=1
    )


alt.FacetChart(...)